# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd 
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
#nltk.download()
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
# load data from database
def load_data(loc='TrialDatabaseName.db',table='messagelabel'):
    engine = create_engine(f'sqlite:///{loc}')
    df = pd.read_sql_table(table_name=table, con=engine)
    X = df['message']
    cols=list(df.columns)
    for item in ['id','message','original','genre'] :
        cols.remove(item)
    Y = df[cols]
    return X, Y

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    words = word_tokenize(text)
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return tokens

In [ ]:
X,Y = load_data()

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix,make_scorer
from sklearn.metrics import classification_report

In [ ]:
def scoring_method(y_pred,y_test):
    return (y_test==y_pred).mean().min()
    
def build_pipeline():
    pipeline = Pipeline([
        ('count',CountVectorizer(tokenizer=tokenize)),
        ('tfid',TfidfTransformer()),
        ('clf',RandomForestClassifier()),
    ])
    
    parameters = {
        'count__ngram_range': ((1, 1), (1, 2)),
        'count__max_df': (0.5, 0.75, 1.0),
        'count__max_features': (None, 5000, 10000),
        'tfid__use_idf': (True, False),
        'clf__n_neighbors': [5,10 ,20,50,100, 200],
        'clf__leaf_size': [5,10, 30, 50,100],
        'clf__weights':['uniform', 'distance'],
        'clf__p': [1,2,3],
    }
    
    cv=GridSearchCV(pipeline,parameters,scoring=make_scorer(scoring_method))
    return cv

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:


def train_pipeline():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model = build_pipeline()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    display_results( y_test, y_pred, cv=model)
    return y_pred,y_test, model

y_pred, y_test, model = train_pipeline()


In [ ]:
def display_results( y_test, y_pred, cv=None):
    print("\nBest Parameters:", cv.best_params_
    accuracy = (y_pred == y_test).mean()
    print('Accuracy Results in each Category \n \n',accuracy)
    print('------------------------')
    for a,b in zip(y_test, y_pred.transpose()):
        confusion_mat = confusion_matrix(y_test[a].values, b)
        print("categories:"+str(a)+'\n',confusion_mat)
        print('\n')

In [ ]:
display_results(y_test,y_pred,cv=model)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
def display_classification( y_test, y_pred):
    for a,b in zip(y_test, y_pred.transpose()):
        print("Classification_report: "+str(a)+'\n',
             classification_report(y_test[a].values,b))
        print('\n')

display_classification(y_test,y_pred)  

Classification_report: related
              precision    recall  f1-score   support

          0       0.61      0.51      0.56      1504
          1       0.86      0.90      0.88      5010
          2       0.22      0.35      0.27        40

avg / total       0.80      0.81      0.80      6554



Classification_report: request
              precision    recall  f1-score   support

          0       0.90      0.98      0.94      5424
          1       0.82      0.47      0.59      1130

avg / total       0.88      0.89      0.88      6554



Classification_report: offer
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554



Classification_report: aid_related
              precision    recall  f1-score   support

          0       0.70      0.91      0.79      3834
          1       0.79      0.45      0.57      2720

avg /

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = parameters = {
        'count__ngram_range': ((1, 1), (1, 2)),
        'count__max_df': (0.5, 0.75, 1.0),
        'count__max_features': (None, 5000, 10000),
        'tfid__use_idf': (True, False),
        'clf__n_neighbors': [5,10 ,20,50,100, 200],
        'clf__leaf_size': [5,10, 30, 50,100],
        'clf__weights':['uniform', 'distance'],
        'clf__p': [1,2,3],
    }

cv = GridSearchCV(pipeline,parameters,scoring=make_scorer(scoring_method))

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
def evaluate_model(model, X_test, Y_test, category_names):
    '''evaluates the performance of model in each label of the category'''
    Y_pred=model.predict(X_test)
    print('--------------------------------')
    print('Evaluation Report')
    print('---------------------------------')
    for a,b,c in zip(Y_test.transpose(), Y_pred.transpose(),category_names):
        print("Category : "+str(c), ' & its result:'+'\n',
             classification_report(a,b))
        print('\n')
    return True


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
import pickle

In [ ]:
def export_model(model,name='trained.model'):
    fp=open(name,'wb')
    fp.write(pickle.dumps(model))
    fp.close()
    

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
%%writefile train.py
import sys
import json
import pandas as pd 
from sqlalchemy import create_engine
import pickle  

#text processing libararies
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt',quiet=True)
nltk.download('stopwords',quiet=True)
nltk.download('wordnet',quiet=True)

#Model building libraries from sklearn ML libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


#important constant
tableName='categorizedMessages'

def load_data(database_filepath,table=tableName):
  '''generates a dataframe for the data present in the database'''
  engine = create_engine(f'sqlite:///{database_filepath}')
  conn=engine.connect()
  df = pd.read_sql_table(table_name=table, con=conn)
  # define features and label arrays
  X = df['message']
  cols=list(df.columns)
  for item in ['id','message','original','genre'] :
      cols.remove(item)
  Y = df[cols].values
  return X, Y, cols 


def tokenize(text):
  '''function to create word tokens for text '''
  text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
  words = word_tokenize(text)
  stop_words = stopwords.words("english")
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
  return tokens

def scoring_method(y_pred,y_test):
    '''function to score results by GridSearch'''
    return (y_test==y_pred).mean().min()

def build_model():
    '''function builds the model in a pipeline structure'''
    pipeline = Pipeline([
      ('count',CountVectorizer(tokenizer=tokenize)),
      ('tfid',TfidfTransformer()),
      ('clf', KNeighborsClassifier()),
    ])
    #  parameters for GridSearchCV
    parameters = {
        'count__ngram_range': ((1, 1), (1, 2)),
        'count__max_df': (0.5, 0.75, 1.0),
        'count__max_features': (None, 5000, 10000),
        'tfid__use_idf': (True, False),
        'clf__n_neighbors': [5,10 ,20,50,100, 200],
        'clf__leaf_size': [5,10, 30, 50,100],
        'clf__weights':['uniform', 'distance'],
        'clf__p': [1,2,3],
    }
    
    # gridsearch object and return as final model pipeline
    pipeline=GridSearchCV(pipeline,
                  parameters,
                  scoring=make_scorer(scoring_method))
    return pipeline



def evaluate_model(model, X_test, Y_test, category_names):
    '''evaluates the performance of model in each label of the category'''
    Y_pred=model.predict(X_test)
    print('--------------------------------')
    print('Evaluation Report')
    print('---------------------------------')
    for a,b,c in zip(Y_test.transpose(), Y_pred.transpose(),category_names):
        print("Category : "+str(c), ' & its result:'+'\n',
             classification_report(a,b))
        print('\n')
    return True


def save_model(model, model_filepath):
    '''helper function to save persistent form of the model'''
    fp=open(model_filepath,'wb')
    pickle.dump(model,fp)
    return True


def main():
    '''main function to run the complete process of building classifier from database's data'''
    if len(sys.argv) == 3:
        database_filepath, model_filepath = sys.argv[1:]
        print('Loading data...\n    DATABASE: {}'.format(database_filepath))
        X, Y, category_names = load_data(database_filepath)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
        
        print('Building model...')
        model = build_model()
        
        print('Training model...')
        model.fit(X_train, Y_train)
        
        print('Evaluating model...')
        print('Best Parameters: ', model.best_params_)
        evaluate_model(model, X_test, Y_test, category_names)

        print('Saving model...\n    MODEL: {}'.format(model_filepath))
        save_model(model, model_filepath)

        print('Trained model saved!')
        model_details = {'model_filepath' :  model_filepath ,
                          'table_name':tableName,
                         'database_filepath': database_filepath,}
        fp=open('models/model_details.txt','w')
        json.dump(model_details,fp)
        fp.close()

    else:
        print('Please provide the filepath of the disaster messages database '\
              'as the first argument and the filepath of the pickle file to '\
              'save the model to as the second argument. \n\nExample: python '\
              'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


if __name__ == '__main__':
    main()


Overwriting train.py


In [ ]:
!python train.py LabelledMessages.db model.pkl

In [ ]:
%%writefile train_classifier.py

import sys
import json
import pandas as pd 
from sqlalchemy import create_engine
import pickle  

#text processing libararies
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt',quiet=True)
nltk.download('stopwords',quiet=True)
nltk.download('wordnet',quiet=True)

#Model building libraries from sklearn ML libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.naive_bayes import ComplementNB
from sklearn.multioutput import MultiOutputClassifier


#important constant
tableName='categorizedMessages'

def load_data(database_filepath,table=tableName):
  '''generates a dataframe for the data present in the database'''
  engine = create_engine(f'sqlite:///{database_filepath}')
  conn=engine.connect()
  df = pd.read_sql_table(table_name=table, con=conn)
  # define features and label arrays
  X = df['message']
  cols=list(df.columns)
  for item in ['id','message','original','genre'] :
      cols.remove(item)
  Y = df[cols].values
  return X, Y, cols 

def tokenize(text):
  '''function to create word tokens for text '''
  text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
  words = word_tokenize(text)
  stop_words = stopwords.words("english")
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
  return tokens

def build_model():
    '''function builds the model in a pipeline structure'''
    pipeline = Pipeline([
      ('count',CountVectorizer(tokenizer=tokenize)),
      ('tfid',TfidfTransformer()),
      ('clf', MultiOutputClassifier(ComplementNB())),
    ])
    #  parameters for GridSearchCV
    parameters = {
        'count__ngram_range': ((1, 1), (1, 2)),
        'count__max_df': (0.1,0.5, 0.75, 1.0),
        'count__max_features': (None, 5000, 10000),
        'tfid__use_idf': (True, False),
        'clf__estimator__alpha': (0.1,0.5,0.8,1.0),
        'clf__estimator__norm': (False,True),
    }

    # gridsearch object and return as final model pipeline
    pipeline=GridSearchCV(pipeline,
                  parameters,)
    return pipeline


def evaluate_model(model, X_test, Y_test, category_names):
    '''evaluates the performance of model in each label of the category'''
    Y_pred=model.predict(X_test)
    print('--------------------------------')
    print('Evaluation Report')
    print('---------------------------------')
    for a,b,c in zip(Y_test.transpose(), Y_pred.transpose(),category_names):
        print("Category : "+str(c), ' & its result:'+'\n',
             classification_report(a,b))
        print('\n')
    return True


def save_model(model, model_filepath):
    '''helper function to save persistent form of the model'''
    fp=open(model_filepath,'wb')
    pickle.dump(model,fp)
    return True


def main():
    '''main function to run the complete process of building classifier from database's data'''
    if len(sys.argv) == 3:
        database_filepath, model_filepath = sys.argv[1:]
        print('Loading data...\n    DATABASE: {}'.format(database_filepath))
        X, Y, category_names = load_data(database_filepath)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
        
        print('Building model...')
        model = build_model()
        
        print('Training model...')
        model.fit(X_train, Y_train)
        
        print('Evaluating model...')
        print('Best Parameters: ', model.best_params_)
        evaluate_model(model, X_test, Y_test, category_names)

        print('Saving model...\n    MODEL: {}'.format(model_filepath))
        save_model(model, model_filepath)

        print('Trained model saved!')
        model_details = {'model_filepath' :  model_filepath ,
                          'table_name':tableName,
                         'database_filepath': database_filepath,}
        fp=open('model_details.txt','w')
        json.dump(model_details,fp)
        fp.close()

    else:
        print('Please provide the filepath of the disaster messages database '\
              'as the first argument and the filepath of the pickle file to '\
              'save the model to as the second argument. \n\nExample: python '\
              'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


if __name__ == '__main__':
    main()

Overwriting train_classifier.py


In [ ]:
!python train_classifier.py DisasterResponse.db classifier.pkl

Loading data...
    DATABASE: DisasterResponse.db
Building model...
Training model...
Evaluating model...
Best Parameters:  {'clf__estimator__alpha': 0.5, 'clf__estimator__norm': False, 'count__max_df': 0.5, 'count__max_features': 10000, 'count__ngram_range': (1, 2), 'tfid__use_idf': True}
--------------------------------
Evaluation Report
---------------------------------
Category : related  & its result:
               precision    recall  f1-score   support

           0       0.55      0.73      0.63      1197
           1       0.91      0.82      0.86      4009

    accuracy                           0.80      5206
   macro avg       0.73      0.78      0.75      5206
weighted avg       0.83      0.80      0.81      5206



Category : request  & its result:
               precision    recall  f1-score   support

           0       0.95      0.85      0.90      4316
           1       0.53      0.79      0.63       890

    accuracy                           0.84      5206
   macr